## Fetching Move Data from the Pokémon API for Damage Simulation

In this notebook, we retrieve detailed information about Pokémon moves by querying the official Pokémon API. The extracted data includes move power, accuracy, type, priority, and damage class, which are essential attributes for accurately simulating battle damage. This step enables us to enrich our dataset with dynamic and up-to-date move characteristics necessary for building a reliable damage calculation model.

### Cleaned dataset loading

In [4]:
import pandas as pd

# =================== Cargar dataset ===================
dat_base = pd.read_csv("../data/pokemon_data_mod.csv")

dat_base.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1302 entries, 0 to 1301
Data columns (total 19 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   id               1302 non-null   int64 
 1   name             1302 non-null   object
 2   base_experience  1302 non-null   int64 
 3   height           1302 non-null   int64 
 4   weight           1302 non-null   int64 
 5   abilities        1302 non-null   object
 6   primary_type     1302 non-null   object
 7   secondary_type   726 non-null    object
 8   move_1           1268 non-null   object
 9   move_2           1265 non-null   object
 10  move_3           1263 non-null   object
 11  move_4           1263 non-null   object
 12  move_5           1262 non-null   object
 13  hp               1302 non-null   int64 
 14  attack           1302 non-null   int64 
 15  defense          1302 non-null   int64 
 16  special-attack   1302 non-null   int64 
 17  special-defense  1302 non-null   

### Getting a Unique Move List from the Database

In [13]:
# =================== Creación de lista de tipos y movimientos ===================
# Crear un set vacío para guardar los movimientos únicos
movimientos_unicos = set()

# Iterar por las columnas de movimientos
for col in ['move_1', 'move_2', 'move_3', 'move_4', 'move_5']:
    # Agregar los valores no nulos de cada columna al set
    movimientos_unicos.update(dat_base[col].dropna().str.strip())

# Crear un set vacío para guardar los tipos únicos
tipos_unicos = set()

# Mostrar resultados de la list
#Movimientos
lista_movimientos_unicos = sorted(list(movimientos_unicos))
print(f"Total unique moves: {len(lista_movimientos_unicos)}\n")
print("List of unique moves:")
print(", ".join(lista_movimientos_unicos))

Total unique moves: 135

List of unique moves:
absorb, acid, acid-armor, agility, amnesia, attract, aurora-beam, bind, bite, blizzard, body-slam, bounce, bubble-beam, bug-bite, charge, charm, comet-punch, confuse-ray, confusion, cosmic-power, counter, cut, defense-curl, destiny-bond, dig, disable, double-edge, double-kick, double-slap, double-team, drill-peck, earthquake, electroweb, ember, fire-punch, fire-spin, flail, flamethrower, fly, fury-attack, growl, growth, guillotine, gust, harden, headbutt, hidden-power, horn-attack, horn-drill, hydro-pump, hyper-beam, ice-beam, ice-punch, infestation, iron-defense, jump-kick, karate-chop, knock-off, leech-seed, leer, light-screen, low-kick, mega-drain, mega-kick, mega-punch, mist, mud-slap, night-shade, pay-day, peck, petal-dance, pin-missile, poison-powder, poison-sting, pound, powder-snow, protect, psybeam, psychic, quick-attack, razor-leaf, razor-wind, recover, recycle, reflect, rest, roar, rock-throw, rolling-kick, rollout, sand-attack,

### Getting Additional Info from the API for Every Unique Move, Including:
- Power
- Accuracy
- PP (Power Points)
- Priority
- Type
- Damage Class

In [16]:
### Sacar info de los movimientos de la API
#pip install requests
import requests
import time

# Tu lista de movimientos únicos (reemplaza con la tuya si ya la tienes)
#lista_movimientos_unicos_prueba = lista_movimientos_unicos[:5]

# Lista para guardar los datos
moves_data = []

for move in lista_movimientos_unicos:
    try:
        url = f"https://pokeapi.co/api/v2/move/{move.strip().lower()}/"
        res = requests.get(url)
        if res.status_code == 200:
            data = res.json()
            move_info = {
                "name": data["name"],
                "power": data["power"],
                "accuracy": data["accuracy"],
                "pp": data["pp"],
                "priority": data["priority"],
                "type": data["type"]["name"],
                "damage_class": data["damage_class"]["name"]
            }
            moves_data.append(move_info)
        else:
            print(f"No encontrado: {move}")
    except Exception as e:
        print(f"Error con {move}: {e}")
    time.sleep(0.2)  # Para no saturar la API

# Convertir a DataFrame
df_moves = pd.DataFrame(moves_data)

In [24]:
import os

if os.path.exists("../data/movimientos_pokemon_info.csv"):
    print("El archivo ya existe.")
else:
    #Guardamos la info obtenida de la api
    df_moves.to_csv("../data/movimientos_pokemon_info.csv", index=False)
    
print(df_moves.head())

El archivo ya existe.
         name  power  accuracy  pp  priority     type damage_class
0      absorb   20.0     100.0  25         0    grass      special
1        acid   40.0     100.0  30         0   poison      special
2  acid-armor    NaN       NaN  20         0   poison       status
3     agility    NaN       NaN  30         0  psychic       status
4     amnesia    NaN       NaN  20         0  psychic       status
